In [1]:
import pandas as pd
import numpy as np

In [4]:
# had to edit first cleaning script to include YearBuilt
ames = pd.read_csv("data/housing_clean_ed2.csv")

In [5]:
# Cut out anything over 4,000 sq ft 
# Drops two outliers
ames.drop(ames[ames.GrLivArea >= 4000].index, inplace=True)

In [6]:
# Change month to str to make categorical
ames.MoSold = ames.MoSold.astype(str)

In [7]:
# Transform ordinal categorical to numerical
ames_rank = ames.filter(regex='Qual$|QC$|Qu$|Cond$').drop(['OverallQual','OverallCond'], axis=1)
ames_rank.replace({'None':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}, inplace=True)
ames[ames_rank.columns] =ames_rank

In [8]:
ames['HouseAge'] = ames.YrSold - ames.YearBuilt

In [9]:
# should we drop this?
ames['RemodelAge'] = ames.YrSold - ames.YearRemodAdd

In [10]:
# deal with some houses do not have GarageYrBlt
ames.GarageYrBlt = ames.GarageYrBlt.replace('None', 100000).astype(float)
ames['GarageAge'] = ames.YrSold - ames.GarageYrBlt
ames.loc[ames.GarageAge < 0, 'GarageAge'] = 'None'

In [11]:
# drop year columns now that we have age features
ames = ames.drop(['YearBuilt', 'YearRemodAdd', 'GarageYrBlt'],axis=1)

In [12]:
# create has Y/N features
ames['HasBsmt'] = np.where(ames['TotalBsmtSF']==0, 0, 1)
ames['HasFireplace'] = np.where(ames['Fireplaces']==0, 0, 1)
ames['HasGarage'] = np.where(ames['GarageArea']==0, 0, 1)
ames['HasWoodDeck'] = np.where(ames['WoodDeckSF']==0, 0, 1)
ames['HasOpenPorch'] = np.where(ames['OpenPorchSF']==0, 0, 1)
ames['HasEncPorch'] = np.where(ames['EnclosedPorch']==0, 0, 1)
ames['Has3SsnPorch'] = np.where(ames['3SsnPorch']==0, 0, 1)
ames['HasScreenPorch'] = np.where(ames['ScreenPorch']==0, 0, 1)
ames['HasPool'] = np.where(ames['PoolArea']==0, 0, 1)
ames['HasFence'] = np.where(ames['Fence']=='None', 0, 1)

In [13]:
ames['HasPorch'] = ames.OpenPorchSF + ames.EnclosedPorch + ames['3SsnPorch'] + ames.ScreenPorch
ames['HasPorch'] = np.where(ames['HasPorch']==0, 0, 1)

In [14]:
# create totsf that includes basement and garage
ames['TotSF'] = ames.GrLivArea + ames.TotalBsmtSF + ames.GarageArea

In [15]:
ames['PriceSqft'] = ames.SalePrice / ames.GrLivArea

In [16]:
# Save csv

ames.to_csv("data\housing_clean2.csv",index_label = False)

In [17]:
ames.head()

,GrLivArea,SalePrice,Utilities,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,...,HasWoodDeck,HasOpenPorch,HasEncPorch,Has3SsnPorch,HasScreenPorch,HasPool,HasFence,HasPorch,TotSF,PriceSqft
0,856,126000,AllPub,SWISU,1Fam,1Story,6,6,Gable,CompShg,...,0,0,0,0,1,0,0,1,2111.0,147.196262
1,1049,139500,AllPub,Edwards,TwnhsE,1Story,5,5,Gable,CompShg,...,0,1,0,0,0,0,0,1,2364.0,132.983794
2,1001,124900,AllPub,IDOTRR,1Fam,1Story,5,9,Hip,CompShg,...,1,0,1,1,0,0,0,1,2054.0,124.775225
3,1039,114000,AllPub,OldTown,1Fam,2Story,4,8,Gable,CompShg,...,0,0,1,0,1,0,0,1,1725.0,109.720885
4,1665,227000,AllPub,NWAmes,1Fam,2Story,8,6,Gable,CompShg,...,0,1,0,0,0,0,0,1,3003.0,136.336336


In [18]:
ames.loc[(ames.OpenPorchSF + ames.EnclosedPorch + ames['3SsnPorch'] + ames.ScreenPorch)==0]

,GrLivArea,SalePrice,Utilities,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,...,HasWoodDeck,HasOpenPorch,HasEncPorch,Has3SsnPorch,HasScreenPorch,HasPool,HasFence,HasPorch,TotSF,PriceSqft
8,889,137500,AllPub,Mitchel,1Fam,1Story,5,6,Gable,CompShg,...,0,0,0,0,0,0,0,0,2237.0,154.668166
11,1274,136000,AllPub,Mitchel,1Fam,2Story,5,5,Gable,CompShg,...,1,0,0,0,0,0,1,0,2474.0,106.750392
19,1091,131250,AllPub,Mitchel,1Fam,SLvl,6,5,Gable,CompShg,...,1,0,0,0,0,0,0,0,1904.0,120.302475
21,789,115000,AllPub,OldTown,1Fam,1Story,5,5,Gable,CompShg,...,0,0,0,0,0,0,0,0,1828.0,145.754119
22,1092,98000,AllPub,MeadowV,Twnhs,2Story,6,9,Gable,CompShg,...,1,0,0,0,0,0,0,0,1638.0,89.743590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2567,1040,129500,AllPub,Sawyer,1Fam,1Story,5,5,Hip,CompShg,...,0,0,0,0,0,0,0,0,2464.0,124.519231
2568,890,115000,AllPub,Sawyer,2fmCon,1Story,5,5,Gable,CompShg,...,0,0,0,0,0,0,0,0,2088.0,129.213483
2569,1242,109900,AllPub,Edwards,1Fam,1Story,4,5,Gable,CompShg,...,0,0,0,0,0,0,0,0,2062.0,88.486312
2573,1733,139600,AllPub,Edwards,1Fam,1Story,3,5,Hip,CompShg,...,0,0,0,0,0,0,0,0,2185.0,80.553953


In [2]:
import pandas as pd


Ames = pd.read_csv("data/Ames_Housing_Price_Data.csv", index_col=0)
Ames = Ames.reset_index(drop = True)
# Garage
Ames = Ames.drop([433,531])
# Basement
Ames.loc[[813,1201],"BsmtExposure"] = "No"
select_column = ["BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","BsmtHalfBath","BsmtFullBath"]
Ames[select_column] = Ames[select_column].fillna(0)
# Masonry Veneer
Ames.MasVnrArea = Ames.MasVnrArea.fillna(0)
# Electric 
Ames.Electrical = Ames.Electrical.fillna(Ames.Electrical.mode()[0])
# Fill others with None
Ames = Ames.fillna("None")
# drop columns and repeative rows
Ames = Ames.drop(1816,axis=0)
drop_columns = ["Alley", "PID", "MSSubClass","MSZoning","LotFrontage","LotArea","Street","LotShape","LandContour",
                      "LotConfig","LandSlope","Condition1","Condition2","SaleType","SaleCondition"]

Ames = Ames.drop(drop_columns,axis=1)
# Save csv
Ames = Ames.reset_index(drop = True)
Ames.to_csv("data\housing_clean_ed2.csv",index_label = False)